In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/My Drive/toxic span/toxic_span_practice.csv" "./toxic_span_practice.csv"
!cp "/content/drive/My Drive/toxic span/toxic_span_train.csv" "./toxic_span_train.csv"
!cp "/content/drive/My Drive/toxic span/bad-words.csv" "./bad-words.csv"

In [3]:
!pip install stanza
!pip install transformers

     |████████████████████████████████| 235kB 5.6MB/s 
     |████████████████████████████████| 1.4MB 5.9MB/s 
     |████████████████████████████████| 2.9MB 26.1MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=aa127ba9cefc28119eda4c5b285dd239e0e4fa26e1c3859e9bfe5945ba08af48
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import pandas as pd
import numpy as np
import gc
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFBertModel,BertTokenizer
import pandas as pd
import textblob
import json
from nltk.tokenize import word_tokenize
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
test_set = pd.read_csv("toxic_span_practice.csv")
test_set['spans'] = test_set['spans'].apply(lambda x : json.loads(x))
train_set = pd.read_csv("toxic_span_train.csv")
train_set['spans'] = train_set['spans'].apply(lambda x : json.loads(x))
toxic_span_dataset = test_set.append(train_set,ignore_index=True)
toxic_span_dataset['text'] = toxic_span_dataset['text'].apply(lambda x : x.lower())

In [7]:
def createNEROutputs(texts,spans,max_length,tokenizer):
    outputs = []
    for text,span in zip(texts,spans):
        output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))
        tokens = tokenizer.tokenize(text)[:max_length]
        length = 0
        start = True
        for i in range(len(tokens),max_length):
            output[i,0] = 1.0
        for index,token in enumerate(tokens):
            sub = False
            if "##" in token:
                sub = True
                token = token[2:]
            if not start:
                next_index = text[length:].find(token)
                if next_index == 0:
                    sub = True
                length += next_index
            # if length in span and not sub:
            #     output[index,2] = 1.0
            #     output[index,0] = 0.0
            if length in span:
                output[index,2] = 1.0
                output[index,0] = 0.0
            else:
                output[index,1] = 1.0
                output[index,0] = 0.0
            length += len(token)
            start = False
        outputs.append(output)
    return np.array(outputs)

In [8]:
def NERGetIndicesSingleText(outputs,text,tokenizer):
    outputs = tf.argmax(outputs,axis=-1)
    tokens = tokenizer.tokenize(text)
    index = 0
    indexes = []
    sub = False
    prev = False
    for token,output in zip(tokens,outputs):
        if token[:2] == "##":
            token = token[2:]
            sub = True
        else:
            sub = False
        temp_index = text[index:].find(token)
        temp_start = index+temp_index
        if output == 2 or (sub and prev and output != 0):
            prev = True
            indexes = indexes + list(range(temp_start,temp_start+len(token)))
        else:
            prev = False
        index = temp_start+len(token)
    return np.array(indexes)

In [9]:
def createIndicesForNERModel(predicts,texts,tokenizer):
    outputs = []
    for text,pred in zip(texts,predicts):
         indices = NERGetIndicesSingleText(pred,text,tokenizer)
         outputs.append(indices)
    return outputs

In [10]:
def f1(preds,trues):
    if len(trues) == 0:
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [11]:
def avg_f1(preds,trues):
    avg_f1_total = 0.0
    for pred,true in zip(preds,trues):
        avg_f1_total += f1(pred,true)
    return avg_f1_total/len(preds)

In [12]:
class F1Metric(callbacks.Callback):
    def __init__(self,inputs,labels,spans,texts,test=True):
        self.inputs = inputs
        self.spans = spans
        self.tokenizer = tokenizer
        self.texts = texts
        self.test = test

    def on_epoch_end(self, epoch, logs={}):
        preds = self.model.predict(self.inputs,verbose=0)
        indices = createIndicesForNERModel(preds,texts,tokenizer)
        f1 = avg_f1(indices,self.spans)
        if self.test:
            print()
            print("test f1 = "+str(f1))
        else:
            print()
            print("train f1 = "+str(f1))

In [13]:
def createInputForNER(texts,max_length,tokenizer):
    input_length = []
    for text in texts:
        input_length.append(min(max_length,len(tokenizer.tokenize(text))))
    tokens = tokenizer(texts,padding="max_length",max_length=max_length,return_tensors="tf",truncation=True)
    data = [np.array(tokens['input_ids']),np.array(tokens['token_type_ids']),np.array(tokens['attention_mask']),np.array(input_length)]
    return data

# CRF Layer

In [14]:
#https://github.com/Hironsan/keras-crf-layer/blob/master/crf.py
import tensorflow as tf
from keras import backend as K
from keras.engine import Layer, InputSpec
import tensorflow_addons as tfa

try:
    from tensorflow.contrib.crf import crf_decode
except ImportError:
    from tensorflow.python.framework import dtypes
    from tensorflow.python.ops import array_ops, gen_array_ops, math_ops, rnn, rnn_cell


    class CrfDecodeForwardRnnCell(rnn_cell.RNNCell):
        def __init__(self, transition_params):
            self._transition_params = array_ops.expand_dims(transition_params, 0)
            self._num_tags = transition_params.get_shape()[0]

        @property
        def state_size(self):
            return self._num_tags

        @property
        def output_size(self):
            return self._num_tags

        def __call__(self, inputs, state, scope=None):
            state = array_ops.expand_dims(state, 2)  # [B, O, 1]
            transition_scores = state + self._transition_params  # [B, O, O]
            new_state = inputs + math_ops.reduce_max(transition_scores, [1])  # [B, O]
            backpointers = math_ops.argmax(transition_scores, 1)
            backpointers = math_ops.cast(backpointers, dtype=dtypes.int32)  # [B, O]
            return backpointers, new_state


    class CrfDecodeBackwardRnnCell(rnn_cell.RNNCell):
        def __init__(self, num_tags):
            self._num_tags = num_tags

        @property
        def state_size(self):
            return 1

        @property
        def output_size(self):
            return 1

        def __call__(self, inputs, state, scope=None):
            state = array_ops.squeeze(state, axis=[1])  # [B]
            batch_size = array_ops.shape(inputs)[0]
            b_indices = math_ops.range(batch_size)  # [B]
            indices = array_ops.stack([b_indices, state], axis=1)  # [B, 2]
            new_tags = array_ops.expand_dims(
                gen_array_ops.gather_nd(inputs, indices),  # [B]
                axis=-1)  # [B, 1]

            return new_tags, new_tags


    def crf_decode(potentials, transition_params, sequence_length):
        num_tags = potentials.get_shape()[2]

        # Computes forward decoding. Get last score and backpointers.
        crf_fwd_cell = CrfDecodeForwardRnnCell(transition_params)
        initial_state = array_ops.slice(potentials, [0, 0, 0], [-1, 1, -1])
        initial_state = array_ops.squeeze(initial_state, axis=[1])  # [B, O]
        inputs = array_ops.slice(potentials, [0, 1, 0], [-1, -1, -1])  # [B, T-1, O]
        backpointers, last_score = rnn.dynamic_rnn(
            crf_fwd_cell,
            inputs=inputs,
            sequence_length=sequence_length - 1,
            initial_state=initial_state,
            time_major=False,
            dtype=dtypes.int32)  # [B, T - 1, O], [B, O]
        backpointers = gen_array_ops.reverse_sequence(backpointers, sequence_length - 1, seq_dim=1)  # [B, T-1, O]

        # Computes backward decoding. Extract tag indices from backpointers.
        crf_bwd_cell = CrfDecodeBackwardRnnCell(num_tags)
        initial_state = math_ops.cast(math_ops.argmax(last_score, axis=1), dtype=dtypes.int32)  # [B]
        initial_state = array_ops.expand_dims(initial_state, axis=-1)  # [B, 1]
        decode_tags, _ = rnn.dynamic_rnn(
            crf_bwd_cell,
            inputs=backpointers,
            sequence_length=sequence_length - 1,
            initial_state=initial_state,
            time_major=False,
            dtype=dtypes.int32)  # [B, T - 1, 1]
        decode_tags = array_ops.squeeze(decode_tags, axis=[2])  # [B, T - 1]
        decode_tags = array_ops.concat([initial_state, decode_tags], axis=1)  # [B, T]
        decode_tags = gen_array_ops.reverse_sequence(decode_tags, sequence_length, seq_dim=1)  # [B, T]

        best_score = math_ops.reduce_max(last_score, axis=1)  # [B]
        return decode_tags, best_score


class CRFLayer(Layer):

    def __init__(self, transition_params=None, **kwargs):
        super(CRFLayer, self).__init__(**kwargs)
        self.transition_params = transition_params
        self.input_spec = [InputSpec(ndim=3), InputSpec(ndim=2)]
        self.supports_masking = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape[0]) == 3

        return input_shape[0]

    def build(self, input_shape):
        assert len(input_shape) == 2
        assert len(input_shape[0]) == 3
        assert len(input_shape[1]) == 2
        n_steps = input_shape[0][1]
        n_classes = input_shape[0][2]
        assert n_steps is None or n_steps >= 2

        self.transition_params = self.add_weight(shape=(n_classes, n_classes),
                                                 initializer='uniform',
                                                 name='transition')
        self.input_spec = [InputSpec(dtype=K.floatx(), shape=(None, n_steps, n_classes)),
                           InputSpec(dtype='int32', shape=(None, 1))]
        self.built = True

    def viterbi_decode(self, potentials, sequence_length):
        decode_tags, best_score = crf_decode(potentials, self.transition_params, sequence_length)
        return decode_tags

    def call(self, inputs, mask=None, **kwargs):
        inputs, sequence_lengths = inputs
        self.sequence_lengths = K.flatten(sequence_lengths)
        y_pred = self.viterbi_decode(inputs, self.sequence_lengths)
        nb_classes = self.input_spec[0].shape[2]
        y_pred_one_hot = K.one_hot(y_pred, nb_classes)

        return K.in_train_phase(inputs, y_pred_one_hot)

    def loss(self, y_true, y_pred):
        y_true = K.cast(K.argmax(y_true, axis=-1), dtype='int32')
        log_likelihood, self.transition_params = tfa.text.crf.crf_log_likelihood(
            y_pred, y_true, self.sequence_lengths, self.transition_params)
        loss = tf.reduce_mean(-log_likelihood)

        return loss

    def get_config(self):
        config = {
            'transition_params': K.eval(self.transition_params),
        }
        base_config = super(CRFLayer, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))


def create_custom_objects():
    instanceHolder = {'instance': None}

    class ClassWrapper(CRFLayer):
        def __init__(self, *args, **kwargs):
            instanceHolder['instance'] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)

    def loss(*args):
        method = getattr(instanceHolder['instance'], 'loss')
        return method(*args)

    return {'CRFLayer': ClassWrapper, 'loss': loss}

# NER model

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# base_model = TFBertModel.from_pretrained('bert-base-uncased')

In [16]:
def createToxicModelWithGivenBaseModel(max_input_length,base_model):
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_type_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_token_type_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    input_length = layers.Input(shape=(1,),name="length",dtype=tf.int32)
    base_model.trainable = True
    base_model = base_model(input_ids_layer,token_type_ids=input_type_ids_layer,attention_mask=input_attention_mask_layer,return_dict=True)
    output = layers.Dropout(0.1)(base_model.last_hidden_state)
    output = layers.Dense(1024,activation="relu")(output)
    output = layers.Dropout(0.1)(output)
    output = layers.Dense(1024,activation="relu")(output)
    output = layers.Dense(3,activation="linear")(output)
    crf = CRFLayer()
    output = crf(inputs=[output,input_length])
    model = models.Model(inputs=[input_ids_layer,input_type_ids_layer,input_attention_mask_layer,input_length],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=3e-5),loss=crf.loss,metrics=['accuracy'])
    return model

In [17]:
max_length = 400

# train on random part of dataset to save a check point

In [18]:
toxic_span_dataset = toxic_span_dataset.sample(frac=1)
texts = toxic_span_dataset['text'].to_numpy()
targets = createNEROutputs(texts,toxic_span_dataset['spans'],max_length,tokenizer)
all_spans = toxic_span_dataset['spans'].to_numpy()
kf = KFold(n_splits=5,shuffle=True)
train_test_indices = []
for train_index,test_index in kf.split(texts):
    train_test_indices.append((train_index,test_index))

In [21]:
train_index,test_index = train_test_indices.pop()
x_train , x_test = list(texts[train_index]) , list(texts[test_index])
y_train , y_test = targets[train_index] , targets[test_index]
model = None
base_model = None
gc.collect()
tf.keras.backend.clear_session()
base_model = TFBertModel.from_pretrained('bert-base-uncased')
model = createToxicModelWithGivenBaseModel(max_length,base_model)
train_data = createInputForNER(x_train,max_length,tokenizer)
test_data = createInputForNER(x_test,max_length,tokenizer)
spans_test = all_spans[test_index]
model.fit(train_data,y_train,batch_size=16,epochs=2)
preds = model.predict(test_data)
indices = createIndicesForNERModel(preds,x_test,tokenizer)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Epoch 1/2
432/432 [==============================] - 736s 2s/step - loss: 11.1398 - accuracy: 0.1117
Epoch 2/2
432/432 [==============================] - 728s 2s/step - loss: 9.1926 - accuracy: 0.1110


In [22]:
hate_words = pd.read_csv("./bad-words.csv")['jigaboo'].to_list()

## tokenize using nltk

In [23]:
indices_all = []
for text,found_indices in zip(x_test,indices):
    start = 0
    indices = []
    for word in word_tokenize(text):
        word_blob = textblob.TextBlob(word)
        if word_blob.polarity < 0 or word in hate_words:
            start_index = text[start:].find(word)
            indices += list(range(start + start_index,start + start_index+len(word)))
            start += start_index+len(word)
    indices_all.append(list(set(indices).union(set(found_indices))))

In [24]:
toxic_f1 = avg_f1(indices_all,spans_test)

In [25]:
print("F1 : %f"%(toxic_f1))

F1 : 0.556994
